# Setup and Imports

In [1]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
from tqdm import tqdm

In [2]:
#functions 
def get_movie_rating(movie_id):
    #movie and release date
    movie = tmdb.Movies(movie_id)
    #create output dict
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] =='US':
            movie_info['certification'] = c['certification']
    return movie_info

def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [3]:
#confirm files from part 1 of project are in Data folder
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2002.csv.gz',
 'final_tmdb_data_2003.csv.gz',
 'final_tmdb_data_2004.csv.gz',
 'final_tmdb_data_2005.csv.gz',
 'final_tmdb_data_2006.csv.gz',
 'final_tmdb_data_2007.csv.gz',
 'final_tmdb_data_2008.csv.gz',
 'final_tmdb_data_2009.csv.gz',
 'final_tmdb_data_2010.csv.gz',
 'final_tmdb_data_2011.csv.gz',
 'final_tmdb_data_2012.csv.gz',
 'final_tmdb_data_2013.csv.gz',
 'final_tmdb_data_2014.csv.gz',
 'final_tmdb_data_2015.csv.gz',
 'final_tmdb_data_2016.csv.gz',
 'final_tmdb_data_2017.csv.gz',
 'final_tmdb_data_2018.csv.gz',
 'final_tmdb_data_2019.csv.gz',
 'final_tmdb_data_2020.csv.gz',
 'final_tmdb_data_2021.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2002.json',
 'tmdb_api_results_2003.json',
 'tmdb_api_results_2004.json',
 'tmdb_api_results_2005.json',
 'tmdb_api_resul

In [4]:
#load basics csv into a dataframe
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama


# API calls

In [5]:
with open('/Users/wombatmaster/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
    
tmdb.API_KEY =  login['api-key']

In [6]:
YEARS_TO_GET = [2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]

For loop to grab the rating for movies from 2000 and 2001


In [7]:
import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)

In [8]:
#start outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    #select JSON file to save progress
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    #create is needed
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
            with open(JSON_FILE,'w') as f:
                json.dump([{'imdb_id':0}],f)
    #save new year             
    df = basics.loc[basics['startYear']==YEAR].copy()
    #save movie ids
    movie_ids = df['tconst'].copy()
    #recall previous results
    previous_df = pd.read_json(JSON_FILE)
    #filter out movies that currently exist in the file
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    # INNER Loop
    for movie_id in tqdm(movie_ids_to_get,
                        desc=f'Movies from {YEAR}',
                        position=1,
                        leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue
            
        final_year_df = pd.read_json(JSON_FILE)
        final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)


YEARS:   0%|          | 0/22 [00:00<?, ?it/s]


Movies from 2000:  56%|█████▋    | 116/206 [00:15<00:12,  7.26it/s]


Movies from 2000: 100%|██████████| 206/206 [00:27<00:00,  7.55it/s]

Movies from 2001:  10%|█         | 25/241 [00:03<00:26,  8.02it/s]


Movies from 2001:  59%|█████▊    | 141/241 [00:18<00:12,  8.14it/s]


Movies from 2001: 100%|██████████| 241/241 [00:31<00:00,  7.69it/s]

Movies from 2002:   5%|▍         | 14/284 [00:01<00:37,  7.27it/s]


...

Movies from 2021:  68%|██████▊   | 774/1141 [01:38<00:45,  8.07it/s]


Movies from 2021:  78%|███████▊  | 887/1141 [01:53<00:34,  7.34it/s]


Movies from 2021:  88%|████████▊ | 1000/1141 [02:08<00:18,  7.80it/s]


Movies from 2021:  97%|█████████▋| 1112/1141 [02:24<00:03,  7.73it/s]


Movies from 2021: 100%|██████████| 1141/1141 [02:28<00:00,  7.69it/s]


In [8]:
results_2000 = pd.read_csv("Data/final_tmdb_data_2000.csv.gz")
results_2000.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.500,22.0,NaN
2,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,5.100,8.0,NaN
3,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,0.000,0.0,NaN
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.wkw-inthemoodforlove.com/,843.0,cn,花樣年華,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.106,1925.0,PG


In [ ]:
results_2001 = pd.read_csv("Data/final_tmdb_data_2001.csv.gz")
results_2001.head()

# Exploratory Data Analysis
  > Once you have your data from the API, they would like you to perform some light EDA to show:

## Load in your csv.gz's of results for each year extracted.
        Concatenate the data into 1 dataframe for the remainder of the analysis.
  

In [24]:
combined_results = pd.concat([results_2000,results_2001])
combined_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2408 entries, 0 to 1203
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                2408 non-null   object 
 1   adult                  2406 non-null   float64
 2   backdrop_path          1254 non-null   object 
 3   belongs_to_collection  220 non-null    object 
 4   budget                 2406 non-null   float64
 5   genres                 2406 non-null   object 
 6   homepage               132 non-null    object 
 7   id                     2406 non-null   float64
 8   original_language      2406 non-null   object 
 9   original_title         2406 non-null   object 
 10  overview               2358 non-null   object 
 11  popularity             2406 non-null   float64
 12  poster_path            2118 non-null   object 
 13  production_companies   2406 non-null   object 
 14  production_countries   2406 non-null   object 
 15  rele

  ## How many movies had at least some valid financial information (values > 0 for budget OR revenue)?
            Please exclude any movies with 0's for budget AND revenue from the remaining visualizations.
        

In [19]:
fin_info = combined_results.loc[(combined_results['budget']>0) & (combined_results['revenue'])>0]
fin_info

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.wkw-inthemoodforlove.com/,843.0,cn,花樣年華,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.106,1925.0,PG
11,tt0120467,0.0,/knok3mNReKqPTplnnqz7E4dd7mD.jpg,NaN,120000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...",NaN,19085.0,en,Vulgar,...,14904.0,87.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Everyone loves a clown... some more than others.,Vulgar,0.0,5.400,42.0,R
13,tt0120630,0.0,/sPAwM8WxMdXNlqeP4F9DMe9LYoY.jpg,"{'id': 718551, 'name': 'Chicken Run Collection...",45000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",NaN,7443.0,en,Chicken Run,...,224834564.0,84.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,This ain't no chick flick. It's poultry in mot...,Chicken Run,0.0,6.723,4086.0,G
16,tt0120753,0.0,NaN,NaN,8000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,318.0,en,The Million Dollar Hotel,...,105983.0,122.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Everyone has something to hide.,The Million Dollar Hotel,0.0,5.942,252.0,R
17,tt0120755,0.0,/24DZfupDlhXeTchmcOkoGRhP5Vg.jpg,"{'id': 87359, 'name': 'Mission: Impossible Col...",125000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",NaN,955.0,en,Mission: Impossible II,...,546388105.0,123.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Expect the impossible again.,Mission: Impossible II,0.0,6.107,5395.0,PG-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,tt0262432,0.0,/5ZVRnltdsQXrRWhYCw7zHU4lruz.jpg,NaN,42000.0,"[{'id': 18, 'name': 'Drama'}]",NaN,18292.0,en,George Washington,...,241816.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"Down this twisted road, please watch over my s...",George Washington,0.0,6.700,89.0,NaN
955,tt0266308,0.0,/amBvmIshdsSkOtvVIgxl7YSQ9Dg.jpg,"{'id': 16302, 'name': 'Battle Royale Collectio...",4500000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,3176.0,ja,バトル・ロワイアル,...,30600000.0,123.0,"[{'english_name': 'Japanese', 'iso_639_1': 'ja...",Released,Could you kill your best friend?,Battle Royale,0.0,7.307,2726.0,NR
957,tt0266408,0.0,/r7zkELMYtf8dSh8nnzwXIH4DFh1.jpg,NaN,13500000.0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,34856.0,zh,雷霆戰警,...,145969.0,92.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Extortion. Murder. Revenge.,China Strike Force,0.0,4.900,22.0,R
1009,tt0274716,0.0,/u5UOWZbNUfCf9Se4Y7Z0g7GqQoC.jpg,"{'id': 997747, 'name': 'Petualangan Sherina Co...",250000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",NaN,199324.0,id,Petualangan Sherina,...,1200000.0,111.0,"[{'english_name': 'Indonesian', 'iso_639_1': '...",Released,Will You Help Your Enemy if He’s in Danger?,Sherina's Adventure,0.0,8.000,4.0,NaN


>There are 328 movies that have values in the budget and revenue columns

## How many movies are there in each of the certification categories (G/PG/PG-13/R)?
       

In [25]:
fin_info['certification'].value_counts()

R        126
PG-13    110
PG        30
G         16
NR         8
Name: certification, dtype: int64

## What is the average revenue per certification category?
        

In [31]:
fin_info.groupby(['certification'])['revenue'].mean().round()

certification
G        105343104.0
NR        22330801.0
PG        91943749.0
PG-13    112365450.0
R         53203545.0
Name: revenue, dtype: float64

## What is the average budget per certification category?

In [30]:
fin_info.groupby(['certification'])['budget'].mean().round()

certification
G        45000000.0
NR       18575000.0
PG       54210000.0
PG-13    48263636.0
R        24759897.0
Name: budget, dtype: float64

# Saving Dataframe to meet customers deliverable request

In [33]:
fin_info.to_csv(r'Data/tmdb_results_combined.csv.gz')